In [17]:
# Cargamos librerías
import pandas as pd
import os
import numpy as np
import weightedcalcs as wc
import gc
import pyreadstat
import re  # Necesaria para la limpieza de nombres

def optimizar_tipos_datos(df):
    """Optimiza los tipos de datos para reducir uso de memoria"""
    for col in df.columns:
        if df[col].dtype == 'float64':
            # Convertir a float32 si es posible
            try:
                df[col] = df[col].astype('float32')
            except:
                # Si hay valores que no se pueden convertir, mantener float64
                pass
        elif df[col].dtype == 'int64':
            # Encontrar el tipo entero más pequeño posible
            try:
                col_min = df[col].min()
                col_max = df[col].max()
                
                if col_min >= 0:
                    if col_max < 255:
                        df[col] = df[col].astype('uint8')
                    elif col_max < 65535:
                        df[col] = df[col].astype('uint16')
                    elif col_max < 4294967295:
                        df[col] = df[col].astype('uint32')
                else:
                    if col_min > -128 and col_max < 127:
                        df[col] = df[col].astype('int8')
                    elif col_min > -32768 and col_max < 32767:
                        df[col] = df[col].astype('int16')
                    elif col_min > -2147483648 and col_max < 2147483647:
                        df[col] = df[col].astype('int32')
            except:
                # Si hay problemas en la conversión, mantener el tipo original
                pass
    
    return df

def clean_column_names(df):
    """Limpia nombres de columnas problemáticos"""
    clean_columns = {}
    for col in df.columns:
        clean_name = str(col).strip()
        clean_name = re.sub(r'[^a-zA-Z0-9_]', '_', clean_name)
        if clean_name and not clean_name[0].isalpha():
            clean_name = 'col_' + clean_name
        clean_columns[col] = clean_name
    
    return df.rename(columns=clean_columns)

def cargar_archivo_stata(ruta_archivo):
    """
    Intenta cargar un archivo Stata con múltiples estrategias
    """
    strategies = [
        # Estrategia 1: Pandas normal
        lambda: pd.read_stata(ruta_archivo, convert_categoricals=False),
        
        # Estrategia 2: Pandas con diferentes opciones
        lambda: pd.read_stata(ruta_archivo, convert_categoricals=True),
        
        # Estrategia 3: Pyreadstat
        lambda: pyreadstat.read_dta(ruta_archivo)[0],
        
        # Estrategia 4: Pyreadstat con limpieza de nombres
        lambda: clean_column_names(pyreadstat.read_dta(ruta_archivo)[0]),
    ]
    
    for i, strategy in enumerate(strategies, 1):
        try:
            df = strategy()
            print(f"✓ Estrategia {i} exitosa para {os.path.basename(ruta_archivo)}")
            return df
        except Exception as e:
            print(f"✗ Estrategia {i} falló: {e}")
            continue
    
    raise Exception(f"Todas las estrategias fallaron para {ruta_archivo}")

def cargar_archivo_sumaria(ruta_archivo):
    """
    Función especial para cargar archivos sumaria problemáticos
    """
    try:
        # Intentar con pyreadstat primero
        df, meta = pyreadstat.read_dta(ruta_archivo)
        
        # Limpiar nombres de columnas
        original_columns = df.columns.tolist()
        clean_columns = {}
        
        for i, col in enumerate(original_columns):
            # Para archivos sumaria, a veces es mejor renombrar completamente
            if not col.strip() or any(c in str(col) for c in ['\x00', '\xff', '\xfe']):
                new_name = f'col_{i}'
            else:
                new_name = col.strip()
                new_name = re.sub(r'[^a-zA-Z0-9_]', '_', new_name)
                if new_name[0].isdigit():
                    new_name = 'col_' + new_name
            
            clean_columns[col] = new_name
        
        df = df.rename(columns=clean_columns)
        return df
        
    except Exception as e:
        print(f"✗ Error cargando sumaria: {e}")
        
        # Intentar método alternativo: leer como binario y reparar
        try:
            with open(ruta_archivo, 'rb') as f:
                content = f.read()
            
            # Buscar y reemplazar caracteres problemáticos
            content = content.replace(b'\x00', b'_')
            content = content.replace(b'\xff', b'_')
            content = content.replace(b'\xfe', b'_')
            
            # Guardar temporalmente
            temp_path = ruta_archivo + '_temp.dta'
            with open(temp_path, 'wb') as f:
                f.write(content)
            
            # Intentar cargar el archivo reparado
            df, meta = pyreadstat.read_dta(temp_path)
            os.remove(temp_path)
            
            return df
            
        except Exception as e2:
            print(f"✗ Error definitivo con sumaria: {e2}")
            raise

def cargar_y_unir_modulos(ruta_base, año):
    """
    Carga todos y une los módulos de interés de la ENAHO
    """
    # Lista de posibles nombres de archivos
    modulos_posibles = {
        'vivienda': [f'enaho01-{año}-100.dta', f'ENAHO01-{año}-100.dta'],
        'personas': [f'enaho01-{año}-200.dta', f'ENAHO01-{año}-200.dta'],
        'educacion': [f'enaho01a-{año}-300.dta', f'ENAHO01A-{año}-300.dta'],
        'empleo_ingresos': [f'enaho01a-{año}-500.dta', f'ENAHO01A-{año}-500.dta'],
        'sumarias': [f'sumaria-{año}.dta', f'SUMARIA-{año}.dta']
    }
    
    dfs = {}
    for nombre, archivos_posibles in modulos_posibles.items():
        archivo_encontrado = None
        for archivo in archivos_posibles:
            ruta_archivo = os.path.join(ruta_base, archivo)
            if os.path.exists(ruta_archivo):
                archivo_encontrado = archivo
                break
        
        if archivo_encontrado is None:
            raise FileNotFoundError(f"No se encontró ningún archivo para {nombre} en {ruta_base}")
        
        ruta_completa = os.path.join(ruta_base, archivo_encontrado)
        
        # Manejar archivos sumaria de manera especial
        if nombre == 'sumarias':
            try:
                dfs[nombre] = cargar_archivo_sumaria(ruta_completa)
            except:
                print("Intentando cargar sumaria como último recurso...")
                # Último recurso: intentar con diferentes configuraciones
                try:
                    dfs[nombre] = pd.read_stata(ruta_completa, convert_categoricals=False, preserve_dtypes=True)
                except:
                    try:
                        dfs[nombre] = pd.read_stata(ruta_completa, convert_categoricals=True)
                    except Exception as e:
                        raise Exception(f"No se pudo cargar el archivo sumaria: {e}")
        else:
            dfs[nombre] = cargar_archivo_stata(ruta_completa)
        
        dfs[nombre] = optimizar_tipos_datos(dfs[nombre])

    # Hacer merge hogares + sumarias (nivel hogar)
    df_hogar = pd.merge(dfs['vivienda'],
                        dfs['sumarias'],
                        on=['conglome', 'vivienda', 'hogar'],
                        how='inner',
                        validate='1:1',
                        suffixes=('_viv', '_sum'))

    # Hacer merge hogares + personas
    df_base = pd.merge(df_hogar,
                       dfs['personas'],
                       on=['conglome', 'vivienda', 'hogar'],
                       how='inner',
                       validate='1:m',
                       suffixes=('_hog', '_per'))

    # Hacer merge personas + educación
    df_base = pd.merge(df_base,
                       dfs['educacion'],
                       on=['conglome', 'vivienda', 'hogar', 'codperso'],
                       how='left')

    # Hacer merge personas + educación + empleo
    df_base = pd.merge(df_base,
                       dfs['empleo_ingresos'],
                       on=['conglome', 'vivienda', 'hogar', 'codperso'],
                       how='left')
    
    return df_base

def crear_variables_geograficas(df):
    """
    Crea variables geográficas estandarizadas
    """
    # Departamento
    df['cod_dpto'] = df['ubigeo_sum'].str[:2]
    nombre_dpto = {
        '01': 'Amazonas', '02': 'Áncash', '03': 'Apurímac', '04': 'Arequipa',
        '05': 'Ayacucho', '06': 'Cajamarca', '07': 'Callao', '08': 'Cusco',
        '09': 'Huancavelica', '10': 'Huánuco', '11': 'Ica', '12': 'Junín',
        '13': 'La Libertad', '14': 'Lambayeque', '15': 'Lima', '16': 'Loreto',
        '17': 'Madre de Dios', '18': 'Moquegua', '19': 'Pasco', '20': 'Piura',
        '21': 'Puno', '22': 'San Martín', '23': 'Tacna', '24': 'Tumbes',
        '25': 'Ucayali'
    }
    df['dpto'] = df['cod_dpto'].map(nombre_dpto)

    # Área
    df['área'] = np.where(
        df['estrato_viv'] <= 5, 1,
        np.where((df['estrato_viv'] >= 6) & (df['estrato_viv'] <= 8), 2, np.nan)
    )
    df['área'] = df['área'].map({1: 'Urbano', 2: 'Rural'})

    # Región natural
    df['regnat'] = np.select(
        [
            df['dominio_viv'].isin([1, 2, 3, 8]),
            df['dominio_viv'].isin([4, 5, 6]),
            df['dominio_viv'] == 7
        ], [1, 2, 3], default=np.nan
    )
    df['regnat'] = df['regnat'].map({1: 'Costa', 2: 'Sierra', 3: 'Selva'})
    
    return df

def calcular_tasa_asistencia_edu_inicial(df, año):
    """
    Calcula la tasa de asistencia de educación inicial (3 a 5 años)
    y lo desagrega por área, región natural y departamento.
    """
    # Convertir variables numéricas
    variables_nums = ['mes_viv', 'p308a', 'p307', 'p203_x', 'p204_x', 'p208a_x']
    for var in variables_nums:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    # Filtra la población de 3 a 5 años
    condicion = (
        (df['codinfor_x'] != "00") &
        (df['p204_x'] == 1) &
        (~df['p203_x'].isin([8, 9])) &
        (df['mes_viv'] >= 4) &
        (df['p208a_x'].between(3, 5))
    )
    df3a5 = df[condicion].copy()
    df3a5['tna_ini'] = 0
    condicion_asistencia = (df3a5['p307'] == 1) & (df3a5['p308a'] == 1)
    df3a5.loc[condicion_asistencia, 'tna_ini'] = 1

    # Calcular resultados con weightedcalcs
    calc = wc.Calculator('factora07')

    # Total mean
    media_total = calc.mean(df3a5, 'tna_ini')

    # Means by area
    media_area = df3a5.groupby('área').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()

    # Means by department (total)
    media_dpto_total = df3a5.groupby('dpto').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()

    # Means by department and area (nested)
    media_dpto_area = df3a5.groupby(['dpto', 'área']).apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()

    # Combine results
    resultados = {
        'año': año,
        'media_total': media_total,
        'media_area': media_area,
        'media_dpto_total': media_dpto_total,
        'media_dpto_area': media_dpto_area,
    }

    return resultados, df3a5

# Configuración principal
RUTA_BASE = r"D:\Mateo\ICSI\ENAHO"
AÑOS = [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 
        2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004]

resultados_anuales = {}

for año in AÑOS:
    print(f"\n{'='*50}")
    print(f"Procesando año: {año}")
    print(f"{'='*50}")
    
    try:
        # Cargar y preparar datos
        ruta_año = os.path.join(RUTA_BASE, str(año), "DTA")
        if not os.path.exists(ruta_año):
            print(f"✗ No existe la ruta: {ruta_año}")
            continue
            
        df_base = cargar_y_unir_modulos(ruta_año, año)
        df_base = crear_variables_geograficas(df_base)
        
        # Calcular indicador
        resultados, df3a5 = calcular_tasa_asistencia_edu_inicial(df_base, año)
        resultados_anuales[año] = resultados
        print(f"✓ Procesado exitosamente año {año}")
        
        # Liberar memoria
        del df_base, df3a5
        gc.collect()
        
    except Exception as e:
        print(f"✗ Error procesando año {año}: {e}")
        continue

# Procesar resultados
if resultados_anuales:
    datos_long_totales = []
    nombre_variable = "Tasa Neta de Asistencia (%) "

    for anio, resultado in resultados_anuales.items():
        # 1. Media total Nacional
        datos_long_totales.append({
            'variable': nombre_variable,
            'dpto': 'Nacional',
            'análisis': 'Total',
            'desagregación': 'Total',
            'año': anio,
            'valor': resultado['media_total']
        })

        # 2. Medias por área a nivel Nacional
        for area, valor in resultado['media_area'].items():
            datos_long_totales.append({
                'variable': nombre_variable,
                'dpto': 'Nacional',
                'análisis': 'Área',
                'desagregación': area,
                'año': anio,
                'valor': valor
            })

        # 3. Medias totales por departamento
        for dpto, valor in resultado['media_dpto_total'].items():
            datos_long_totales.append({
                'variable': nombre_variable,
                'dpto': dpto,
                'análisis': 'Total',
                'desagregación': 'Total',
                'año': anio,
                'valor': valor
            })

        # 4. Medias por departamento y área
        for (dpto, area), valor in resultado['media_dpto_area'].items():
            datos_long_totales.append({
                'variable': nombre_variable,
                'dpto': dpto,
                'análisis': 'Área',
                'desagregación': area,
                'año': anio,
                'valor': valor
            })

    # Convertir a DataFrame
    df_resultados = pd.DataFrame(datos_long_totales)
    df_resultados['valor'] = df_resultados['valor'] * 100
    
    # Ordenar resultados
    tna_ini = df_resultados.sort_values(by=['año', 'dpto', 'análisis', 'desagregación'], 
                                      ascending=[False, True, True, True])
    
    print("\n✓ Procesamiento completado exitosamente!")
    print(f"Total de registros procesados: {len(tna_ini)}")
    
    # Mostrar primeras filas
    print("\nPrimeras 10 filas del resultado:")
    print(tna_ini.head(10))
    
else:
    print("✗ No se procesaron datos de ningún año")


Procesando año: 2024
✓ Estrategia 1 exitosa para enaho01-2024-100.dta
✓ Estrategia 1 exitosa para enaho01-2024-200.dta
✓ Estrategia 1 exitosa para enaho01a-2024-300.dta
✓ Estrategia 1 exitosa para enaho01a-2024-500.dta


C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:284: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_area = df3a5.groupby('área').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:287: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_dpto_total = df3a5.groupby('dpto').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo

✓ Procesado exitosamente año 2024

Procesando año: 2023
✓ Estrategia 1 exitosa para enaho01-2023-100.dta
✓ Estrategia 1 exitosa para enaho01-2023-200.dta
✓ Estrategia 1 exitosa para enaho01a-2023-300.dta
✓ Estrategia 1 exitosa para enaho01a-2023-500.dta


C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:284: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_area = df3a5.groupby('área').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:287: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_dpto_total = df3a5.groupby('dpto').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo

✓ Procesado exitosamente año 2023

Procesando año: 2022
✓ Estrategia 1 exitosa para enaho01-2022-100.dta
✓ Estrategia 1 exitosa para enaho01-2022-200.dta
✓ Estrategia 1 exitosa para enaho01a-2022-300.dta
✓ Estrategia 1 exitosa para enaho01a-2022-500.dta


...

✓ Procesado exitosamente año 2014

Procesando año: 2013
✓ Estrategia 1 exitosa para enaho01-2013-100.dta
✓ Estrategia 1 exitosa para enaho01-2013-200.dta
✓ Estrategia 1 exitosa para enaho01a-2013-300.dta
✓ Estrategia 1 exitosa para enaho01a-2013-500.dta


C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:284: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_area = df3a5.groupby('área').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:287: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_dpto_total = df3a5.groupby('dpto').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo

✓ Procesado exitosamente año 2013

Procesando año: 2012
✓ Estrategia 1 exitosa para enaho01-2012-100.dta
✓ Estrategia 1 exitosa para enaho01-2012-200.dta
✓ Estrategia 1 exitosa para enaho01a-2012-300.dta
✓ Estrategia 1 exitosa para enaho01a-2012-500.dta


C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:284: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_area = df3a5.groupby('área').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo\AppData\Local\Temp\ipykernel_25428\963258145.py:287: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  media_dpto_total = df3a5.groupby('dpto').apply(lambda x: calc.mean(x, 'tna_ini')).to_dict()
C:\Users\mateo

✓ Procesado exitosamente año 2012

Procesando año: 2011
✓ Estrategia 1 exitosa para enaho01-2011-100.dta
✓ Estrategia 1 exitosa para enaho01-2011-200.dta
✓ Estrategia 1 exitosa para enaho01a-2011-300.dta
✓ Estrategia 1 exitosa para enaho01a-2011-500.dta
✗ Error cargando sumaria: Unable to read from file
✗ Error definitivo con sumaria: Unable to read from file
Intentando cargar sumaria como último recurso...
✗ Error procesando año 2011: No se pudo cargar el archivo sumaria: unpack requires a buffer of 1 bytes

Procesando año: 2010
✓ Estrategia 1 exitosa para enaho01-2010-100.dta
✓ Estrategia 1 exitosa para enaho01-2010-200.dta
✓ Estrategia 1 exitosa para enaho01a-2010-300.dta
✗ Error procesando año 2010: No se encontró ningún archivo para empleo_ingresos en D:\Mateo\ICSI\ENAHO\2010\DTA

Procesando año: 2009
✗ No existe la ruta: D:\Mateo\ICSI\ENAHO\2009\DTA

Procesando año: 2008
✗ No existe la ruta: D:\Mateo\ICSI\ENAHO\2008\DTA

Procesando año: 2007
✗ No existe la ruta: D:\Mateo\ICSI\ENA